# Naive Bayes Classification with CR Dataset
<hr>

We will build a text classification model using Naive Bayes on the Customer Reviews Dataset. Since there is no standard train/test split for this dataset, we will use 10-Fold Cross Validation (CV). 

## Load the library

In [3]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
import random
from nltk.corpus import stopwords, twitter_samples
# from nltk.tokenize import TweetTokenizer
from sklearn.model_selection import KFold
from nltk.stem import PorterStemmer
from string import punctuation
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
import time

%config IPCompleter.greedy=True
%config IPCompleter.use_jedi=False
# nltk.download('twitter_samples')

## Load the Dataset

In [4]:
corpus = pd.read_pickle('../0_data/CR/CR.pkl')
corpus.label = corpus.label.astype(int)
print(corpus.shape)
corpus

(3775, 3)


,sentence,label,split
0,weaknesses are minor the feel and layout of th...,0,train
1,many of our disney movies do n 't play on this...,0,train
2,player has a problem with dual layer dvd 's su...,0,train
3,i know the saying is you get what you pay for ...,0,train
4,will never purchase apex again .,0,train
...,...,...,...
3770,"so far , the anti spam feature seems to be ver...",1,train
3771,i downloaded a trial version of computer assoc...,1,train
3772,i did not have any of the installation problem...,1,train
3773,their products have been great and have saved ...,1,train


In [5]:
corpus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3775 entries, 0 to 3774
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sentence  3775 non-null   object
 1   label     3775 non-null   int32 
 2   split     3775 non-null   object
dtypes: int32(1), object(2)
memory usage: 73.9+ KB


In [6]:
corpus.groupby( by='label').count()

,sentence,split
label,,
0,1368,1368
1,2407,2407


In [7]:
# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

## Raw Number of Vocabulary

In [8]:
# Build the raw vocobulary for first inspection
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)
vocab_raw = tokenizer.word_index
print('\nThe vocabulary size: {}\n'.format(len(vocab_raw)))
print(vocab_raw)


The vocabulary size: 5334

{'the': 1, 'and': 2, 'i': 3, 'it': 4, 'to': 5, 'a': 6, 'is': 7, 'of': 8, 'this': 9, 'with': 10, 'for': 11, 'you': 12, 'that': 13, 'in': 14, 'have': 15, 'but': 16, 'not': 17, 'on': 18, 'my': 19, "'t": 20, 'as': 21, "'s": 22, 'are': 23, 'very': 24, 'can': 25, 'was': 26, 'use': 27, 'camera': 28, 'player': 29, 'all': 30, 'has': 31, 'phone': 32, 'great': 33, 'one': 34, 'be': 35, 'so': 36, 'at': 37, 'if': 38, 'good': 39, 'easy': 40, 'quality': 41, 'or': 42, 'up': 43, 'from': 44, 'no': 45, 'like': 46, 'an': 47, 'than': 48, 'had': 49, 'just': 50, 'battery': 51, 'get': 52, 'about': 53, 'when': 54, 'software': 55, 'only': 56, 'your': 57, 'ipod': 58, 'would': 59, 'which': 60, 'well': 61, 'they': 62, 'also': 63, 'more': 64, 'sound': 65, 'there': 66, 'me': 67, 'n': 68, 'other': 69, 'do': 70, 'will': 71, 'out': 72, 'does': 73, 'any': 74, "'ve": 75, 'features': 76, 'its': 77, 'zen': 78, 'router': 79, 'even': 80, 'some': 81, 'really': 82, 'by': 83, 'better': 84, 'after': 85

<!--## Split Dataset-->

# Data Preprocessing
<hr>

## Define `clean_doc` function

In [9]:
from nltk.corpus import stopwords
stopwords = stopwords.words('english')
stemmer = PorterStemmer()
    
def clean_doc(doc):
    # split into tokens by white space
    tokens = doc.split()
    # prepare regex for char filtering
    re_punc = re.compile('[%s]' % re.escape(punctuation))
    # remove punctuation from each word
    tokens = [re_punc.sub('', w) for w in tokens]
    # remove remaining tokens that are not alphabetic
#     tokens = [word for word in tokens if word.isalpha()]
    # filter out stop words
    tokens = [w for w in tokens if not w in stopwords]
    # filter out short tokens
    tokens = [word for word in tokens if len(word) >= 1]
    # Stem the token
    tokens = [stemmer.stem(token) for token in tokens]
    return tokens

## Define `count_docs` function

In [10]:
def count_docs(data, docs, ys):
    '''
    Input:
        data: a dictionary that will be used to map each pair to its frequency
        docs: a list of sentences
        ys: a list corresponding to the sentiment of each tweet (either 0 or 1)
    Output:
        result: a dictionary mapping each pair to its frequency
    '''
    onehot = OneHotEncoder( sparse=False)
    y_onehot = onehot.fit_transform(np.reshape(ys, (-1,1)))
    # print(y_onehot)
    count = 0                    
    for doc, y in zip(docs, y_onehot):
        # For each word
        for word in clean_doc(doc):
            # if not in the data yet
            if word not in data:
                # assign it
                data[word] = y
            # if already in the data 
            else:
                # update it
                data[word] = data.get(word) + y

    return data

## Build Frequencies Dictionary

In [11]:
# Build the freqs dictionary for later uses
freqs = count_docs({}, sentences, labels)
freqs

{'weak': array([10.,  2.]),
 'minor': array([8., 1.]),
 'feel': array([19., 29.]),
 'layout': array([3., 2.]),
 'remot': array([17., 13.]),
 'control': array([34., 50.]),
 'n': array([83., 61.]),
 'show': array([8., 8.]),
 'complet': array([16.,  3.]),
 'file': array([36., 53.]),
 'name': array([9., 4.]),
 'mp3': array([14., 59.]),
 'realli': array([34., 85.]),
 'long': array([25., 31.]),
 'must': array([13., 13.]),
 'cycl': array([2., 1.]),
 'everi': array([22., 18.]),
 'zoom': array([17., 34.]),
 'set': array([25., 62.]),
 '2x': array([2., 1.]),
 '3x': array([2., 6.]),
 '4x': array([1., 5.]),
 '1': array([46., 47.]),
 'etc': array([ 9., 33.]),
 'get': array([126., 117.]),
 'back': array([25., 21.]),
 'normal': array([ 9., 12.]),
 'size': array([18., 58.]),
 'sorri': array([1., 1.]),
 'ignor': array([2., 0.]),
 'way': array([35., 33.]),
 '1x': array([1., 0.]),
 'quickli': array([ 7., 13.]),
 'mani': array([27., 43.]),
 'disney': array([2., 0.]),
 'movi': array([13., 13.]),
 'play': ar

In [12]:
# convert the freqs dictionary to nested list
def freqs_to_df(freqs, train_y):
    '''
    input:
        freqs: a frequencies dictionary (ex: {'simplist': array([15.,  4.]), 
                                              'silli': array([64., 20.]), . . })
        train_y: labels for data
    output:
        a frequencies dictionary in the form of dataframe
    '''
    # initialize an empty list to store the rows for dataframe
    freqs_list = []
    
    # Define the names of the dataframe columns
    column_names = ['word']
    column_names = column_names + list(np.unique(train_y))
    
    # convert the keys from the freqs dictionary to a list
    keys = list(freqs.keys())

    # For each row
    for i in range(len(freqs)):
        
        # define the elements for each column
        row = [keys[i]] + list(freqs.get(keys[i]))
        
        # update the frequency list
        freqs_list.append(row)
    
    # Create the dataframe
    df = pd.DataFrame(freqs_list, columns=column_names)
    df.set_index('word', inplace=True)
    return df

In [13]:
freqs_df = freqs_to_df(freqs, labels)
freqs_df

,0,1
word,,
weak,10.0,2.0
minor,8.0,1.0
feel,19.0,29.0
layout,3.0,2.0
remot,17.0,13.0
...,...,...
pi,0.0,1.0
1ghz,0.0,1.0
rd,0.0,1.0


In [14]:
freqs_df[0].sum()

12725.0

# Training and Testing the Model

## Build Training Function

In [15]:
def train_naive_bayes(freq_df, train_x, train_y):
    '''
    Input:
        freqs: a pandas dataframe with word indexing
        train_x: a list of tweets
        train_y: a list of labels correponding to the tweets (0,1)
    Output:
        logprior: the log prior. (equation 3 above)
        loglikelihood: the log likelihood of you Naive bayes equation. (equation 6 above)
    '''
    
    freqs = freq_df

    # calculate V, the number of unique words in the vocabulary
    vocab = list(freqs.index)
    V = len(vocab)
    
    ########################################################################################
    # Part 1: Calculate the log prior probability for each class
    
    # Calculate D, the number of documents
    D = len(train_y)
    
    labels = list(np.unique(train_y.astype(int)))
    count = np.zeros((len(labels),))

    for train_label in train_y:
        for unique_label in labels:
            if train_label == unique_label:
                count[unique_label]+=1
                
    # -> count = [4000, 4000]; it means perfectly balanced between each classese

    # Calculate prior probability for each class
    prior = count/D # -> prior = array([0.5, 0.5])
    
        
    # Calculate the logprior for each class
    logprior = np.log(prior) # -> prior = array([-0.69314718, -0.69314718])
    
    ########################################################################################
    # Part 2.a. Calculate the total number of word occurrences for each class
    
    columns = list(freqs.columns)
    N_classes = []
    # calculate N frequency for each class
    for column in columns:
        
        freqs[column] = (freqs[column] + 1)/(freqs[column].sum()+V)
        
    # Calculate the log likelihood of the word
    loglikelihood = np.log(freqs)
        
    ########################################################################################
        
    return logprior, loglikelihood

In [16]:
labels = np.array(labels)
logprior, loglikelihood = train_naive_bayes(freqs_df, sentences, labels)
print(logprior)
loglikelihood

[-1.01505056 -0.45001922]


,0,1
word,,
weak,-7.309517,-9.014488
minor,-7.510187,-9.419953
feel,-6.711680,-6.711903
layout,-8.321117,-9.014488
remot,-6.817040,-7.474043
...,...,...
pi,-9.707412,-9.419953
1ghz,-9.707412,-9.419953
rd,-9.707412,-9.419953


## Build Testing Function

In [17]:
def naive_bayes_predict(tweet, logprior, loglikelihood):
    '''
    input:
        tweet: a string
        logprior: initial probability based on dataset
        loglikelihood: a dictionary of words mapping to numbers
    output:
        p: the sum of all the loglikelihood of each word in the tweet
        (if found in the dictionary) + logprior (a number)
    '''
    
    # process the tweet to get the list of words
    words = clean_doc(tweet)
    
    
    # Initialize probability to zero
    probs = []
    columns = list(loglikelihood.columns)
    for column in columns:
        prob = 0
        # Iterate for each word in word list
        for word in words:

            # check if the word exist in the loglikelihood dictionary
            if word in loglikelihood.index:
                prob += loglikelihood.loc[word, column]
        
        probs.append(prob)
        
    probs = logprior + probs
    y_hat = np.argmax(probs)
    
    return probs, y_hat

In [18]:
def test_naive_bayes(test_x, test_y, logprior, loglikelihood):
    """
    input:
        test_x: A list of tweets
        test_y: the corresponding labels for the list of tweets
        logprior: the logprior
        loglikelihood: a dictionary with the loglikelihoods for each word
    output:
        accuracy: (# of tweets classified correctly)/total # of tweets
    """
    # initial accuracy
    acc = 0
    
    # initialize an empty list for storing the predictions
    y_hats = []

    for tweet in test_x:
        
        _ , y = naive_bayes_predict(tweet, logprior, loglikelihood)
        
#         y_hat = np.argmax(probs)
        
        # update the y_hats
        y_hats.append(y)
        
#     Error: the mean absolute values between y_hats and test_y
    error = np.mean(np.abs(np.array(y_hats)-np.array(test_y)))
        
#     Accuracy is 1 - error
    acc = 1-error
    return acc    

## KFold CV

In [19]:
###############################################
# Training and Testing using the same Dataset #
###############################################

# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), np.array(list(corpus.label))

# Build the freqs dictionary for later uses
freqs = count_docs({}, sentences, labels)

# Turn the frequencies dictionary into dataframe
freqs_df = freqs_to_df(freqs, labels)
print(freqs_df.head())

# Retrieve the logprior and loglikelihood
logprior, loglikelihood = train_naive_bayes(freqs_df, sentences, labels)
print(logprior)
print(loglikelihood.head())

print("Naive Bayes accuracy = %0.4f" %
      (test_naive_bayes(sentences, labels, logprior, loglikelihood)))

           0     1
word              
weak    10.0   2.0
minor    8.0   1.0
feel    19.0  29.0
layout   3.0   2.0
remot   17.0  13.0
[-1.01505056 -0.45001922]
               0         1
word                      
weak   -7.309517 -9.014488
minor  -7.510187 -9.419953
feel   -6.711680 -6.711903
layout -8.321117 -9.014488
remot  -6.817040 -7.474043
Naive Bayes accuracy = 0.8903


In [20]:
#######################################################
# Training and Testing using 10-fold Cross Validation #
#######################################################

# prepare cross validation
kfold = KFold(10, True)

# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)
# count = 0
acc_list = []
# kfold.split() will return set indices for each split
for train, test in kfold.split(sentences):
    train_x, train_y = [], []
    test_x, test_y = [], []
    for i in train:
        train_x.append(sentences[i])
        train_y.append(labels[i])
        
    for i in test:
        test_x.append(sentences[i])
        test_y.append(labels[i])
    
    train_y = np.array(train_y)
    test_y = np.array(test_y)
    
    # Build the freqs dictionary for later uses
    freqs = count_docs({}, train_x, train_y)

    # Turn the frequencies dictionary into dataframe
    freqs_df = freqs_to_df(freqs, train_y)
#     print(freqs_df.head())

    # Retrieve the logprior and loglikelihood
    logprior, loglikelihood = train_naive_bayes(freqs_df, train_x, train_y)
    print('loglikelihood:\n{}'.format(loglikelihood.head()))
    print('logprior: {}'.format(logprior))
    
    acc = test_naive_bayes(test_x, test_y, logprior, loglikelihood)
    print("Naive Bayes test accuracy = %0.4f\n" %(acc))
    
    acc_list.append(acc)

acc_list = np.array(acc_list)
print()
print('The test ccuracy for each training:\n{}'.format(acc_list))
print('The mean of the test accuracy: ', acc_list.mean())

C:\Users\Diardano Raihan\Anaconda3\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass shuffle=True as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


loglikelihood:
               0         1
word                      
weak   -7.325412 -8.913775
minor  -7.430773 -9.319240
feel   -6.855409 -6.793511
layout -8.934850 -8.913775
remot  -6.794784 -7.447437
logprior: [-1.00697518 -0.45463804]
Naive Bayes test accuracy = 0.7937

loglikelihood:
               0         1
word                      
weak   -7.310283 -9.316276
minor  -7.415643 -9.316276
feel   -6.617136 -6.677218
layout -8.226573 -8.910811
remot  -6.840279 -7.370366
logprior: [-1.01506584 -0.45001054]
Naive Bayes test accuracy = 0.7831

loglikelihood:
               0          1
word                       
mani   -6.431331  -6.284445
disney -8.510773 -10.022115
movi   -7.124478  -7.383057
n      -5.346705  -5.944577
play   -5.584033  -5.691381
logprior: [-1.02486184 -0.44448566]
Naive Bayes test accuracy = 0.7857

loglikelihood:
               0         1
word                      
weak   -7.316349 -8.920834
minor  -7.539492 -9.326300
feel   -6.674495 -6.618249
layout -8.23264